In [ ]:
from google.colab import drive  
drive.mount('/content/drive')

## Aİ2FC - RETİNANET MODELİ İçin Colab Kullanımı ve Temel Bilgiler

In [ ]:
##Doğrulama İşlemi
""" from google.colab import drive
* drive.mount('/gdrive')
* %cd /gdrive
"""
##Kütüphane Kurulumu
"""
*!pip install - q keras #örnek keras kurulumu
*!apt -get install graphviz -y #bir diğer örnek kütüphane graphviz
*import keras #kütüphane import aşaması
"""
## Donanım Türünün Belirlenmesi
"""
*GPU ve TPU iki seçeneği vardır.
Yukarıda ki 
*Çalışma zamana >> Çalışma zamanı türünü değiştir
*GPU veya tpu seç
"""
##Donanım Türü ve Versiyonunu görmek İçin
"""
import tensorflow_hub as hub
print("TF version:",tf.__version__)
print("Hub version:"+hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "Not avaliable")
"""
##Colabta Kulalnılan GPU modelini Görmek için 
"""
*from tensorflow.python.client import device_lib
*device_lib_list_local_device()
"""
##Python Kodunun Colabta Çalıştırılması
"""
=>Dizin kontrolü yapılır.
*!ls
*Eğer ki colabta yüklediğiniz dosyayla aynı dizindeyseniz
*!python3 dosya-adi.py ile kodu çalıştırın"""
#DAHA FAZLASI İÇİN https://rahul-metangale.medium.com/google-colaboratory-cheat-sheet-24b99813b0f0

In [ ]:
#Dosya Silme Nasıl Yapılır ? 
#!rm -r dosya_adi 

#Çevre Bileşenlerinin Kurulumu Aşamaları
* Seçilen reponun colab üzerine klonlanması (https://github.com/fizyr/keras-retinanet.git)
* Kütüphanelerin Kurulumu ve İmport edilmesi


In [ ]:
!git clone https://github.com/fizyr/keras-retinanet.git

In [ ]:
!rm -r sample_data/

In [ ]:
%cd keras-retinanet/ 
!pip install .

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
import os
import shutil
import zipfile
import urllib.request
import xml.etree.ElementTree as ET
import csv
import numpy as np
import pandas
from google.colab import drive
from google.colab import files

In [ ]:
!pip uninstall keras -y 
!pip uninstall keras-nightly -y 
!pip uninstall keras-Preprocessing -y 
!pip uninstall keras-vis -y 
!pip uninstall tensorflow -y

In [ ]:
s!pip install tensorflow==2.3.0 
!pip install keras==2.4

#Veri Setinin Hazırlanması


In [ ]:
#dosyaların oluşturulması
DATASET_DIR = 'veriseti'
ANNOTATIONS_FILE = 'annotations.csv'
CLASSES_FILE = 'classes.csv'

In [ ]:
#zipin içerisinden çıkarılan dosyanın adının "veriseti" olarak değiştirilmesi gerekiyor
#verisetinin indirilmesi
import shutil
shutil.copy("/content/drive/MyDrive/2020-2021 Yarışması/verisetison.zip","/content/keras-retinanet/")
#os.rename(r'veri',r'veriseti')

In [ ]:
#!rm -rf veriseti

In [ ]:
os.rename(r'verisetison',r'veriseti')

In [ ]:
annotations = []
classes = set([])

for xml_file in [f for f in os.listdir(DATASET_DIR) if f.endswith(".xml")]:
  tree = ET.parse(os.path.join(DATASET_DIR, xml_file))
  root = tree.getroot()

  file_name = None

  for elem in root:
    if elem.tag == 'filename':
      file_name = os.path.join(DATASET_DIR, elem.text)

    if elem.tag == 'object':
      obj_name = None
      coords = []
      for subelem in elem:
        if subelem.tag == 'name':
          obj_name = subelem.text
        if subelem.tag == 'bndbox':
          xmin = ""
          xmax = ""
          ymin = ""
          ymax = ""
          for subsubelem in subelem:
            f = float(subsubelem.text)
            i = int(f)
            if subsubelem.tag == "xmin":
              xmin = str(i)
            if subsubelem.tag == "xmax":
              xmax = str(i)
            if subsubelem.tag == "ymin":
              ymin = str(i)
            if subsubelem.tag == "ymax":
              ymax = str(i)
          
          coords.append(xmin)
          coords.append(ymin)
          coords.append(xmax)
          coords.append(ymax)
            
      item = [file_name] + coords + [obj_name]
      annotations.append(item)
      classes.add(obj_name)

with open(ANNOTATIONS_FILE, 'w') as f:
  writer = csv.writer(f)
  writer.writerows(annotations)

with open(CLASSES_FILE, 'w') as f:
  for i, line in enumerate(classes):

    f.write('{},{}\n'.format(line,i))

#Modelin Eğitilmesi(Training)

In [ ]:
#%cd /content/keras-retinanet/veriseti

In [ ]:
#os.rename(r'3 (141).xml',r'a.xml')

In [ ]:
#!rm a.xml

In [ ]:
!zip -r temp.zip keras-retinanet/ 

In [ ]:
#%cd /content/keras-retinanet

In [ ]:
PRETRAINED_MODEL = './snapshots/_pretrained_model.h5'
URL_MODEL = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
urllib.request.urlretrieve(URL_MODEL, PRETRAINED_MODEL)
print('Downloaded pretrained model to ' + PRETRAINED_MODEL)
!keras_retinanet/bin/train.py --freeze-backbone --random-transform --weights {PRETRAINED_MODEL} --batch-size 10 --steps 700  --epochs 3 csv annotations.csv classes.csv

In [ ]:
#### Modeli Drive Çıkarma ####
drive.mount('/content/gdrive')
COLAB_MODEL = './snapshots/resnet50_csv_10.h5'
DRIVE_DIR = '/content/gdrive/My Drive/Colab Notebooks/'
shutil.copy(COLAB_MODEL, DRIVE_DIR)

# Modelin Google Drive Üzerinden Çekilerek LİNKEDİN Paylaşımı İçin Test Edilmesi [1 kereye mahsus]

In [ ]:
#### Modeli Drivedan çekme ####
!gdown https://drive.google.com/uc?id=1MNgd4wRjNJyxr1XiKvICYgO1y-BiHq4Z
shutil.copy("/content/drive/MyDrive/2020-2021 Yarışması/15epoch_10batch_700steps_18haziran.h5","/content/keras-retinanet/snapshots")

#Çıkarım - Test(İnference)

In [ ]:
import shutil
shutil.copy("/content/drive/MyDrive/2020-2021 Yarışması/17epoch_10batch_700steps_18haziran.h5","/content/keras-retinanet/snapshots")

In [ ]:
THRES_SCORE = 0.4

In [ ]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%reload_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf

Using TensorFlow backend.


In [ ]:
import os
%cd /content/keras-retinanet

/content/keras-retinanet


In [ ]:
!pip install numpy==1.19.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numpy-1.19.5-cp37-cp37m-manylinux2010_x86_64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.2.2 requires numpy>=1.21, but you have numpy 1.19.5 which is incompatible.
tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.19.5 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install pyxdameraulevenshtein==1.5.3
!pip install pandas==1.1.4  
!pip install scikit-learn==0.20.2
!pip install Numpy==1.16.1
#Works well in Python3.6, Issue in Python3.7.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 58 kB 3.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyxdameraulevenshtein: filename=pyxDamerauLevenshtein-1.5.3-cp37-cp37m-linux_x86_64.whl size=88235 sha256=2982bf23fb13fdb3a97eda0f99ae0d60e42454d0b85a8ee8bd049b206fbd01ca
  Stored in directory: /root/.cache/pip/wheels/61/51/91/3fcd99c9ac5d66dee56560ae19d106668c84fa8c2e42e23e7f
Successfully built pyxdameraulevenshtein
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.5 MB 5.0 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5
ERROR: pip's dependency resolver does not currently take into account al

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.4 MB 3.9 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.4 requires scikit-learn>=1.0.0, but you have scikit-learn 0.20.2 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.20.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 17.3 MB 527 kB/s 
  Attempting uninstall: Numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully 

In [ ]:
model_path = os.path.join('snapshots/', sorted(os.listdir('snapshots'), reverse=True)[0])
print(model_path)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

# load label to names mapping for visualization purposes
labels_to_names = pandas.read_csv(CLASSES_FILE,header=None).T.loc[0].to_dict()

[autoreload of numpy.lib failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'type_check' is not defined
]
[autoreload of numpy.matrixlib failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'defmatrix' is not defined
]
[autoreload of numpy.linalg.linalg failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
ValueError: Only callable can be used as callback
]
[autoreload of numpy.testing failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload,

snapshots/_pretrained_model.h5


ValueError: ignored

In [ ]:
def img_inference(img_path):
  image = read_image_bgr(img_infer)

  # copy to draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess image for network
  image = preprocess_image(image)
  image, scale = resize_image(image)

  # process image
  start = time.time()
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
  print("processing time: ", time.time() - start)

  # correct for image scale
  boxes /= scale

  # visualize detections
  for box, score, label in zip(boxes[0], scores[0], labels[0]):
      # scores are sorted so we can break
      if score < THRES_SCORE:
          break

      color = label_color(label)

      b = box.astype(int)
      draw_box(draw, b, color=color)

      caption = "{} {:.3f}".format(labels_to_names[label], score)
      draw_caption(draw, b, caption)

  plt.figure(figsize=(10, 10))
  plt.axis('off')
  plt.imshow(draw)
  plt.show()

In [ ]:
uploaded = files.upload()
img_infer = list(uploaded)[0]

print('Running inference on: ' + img_infer)
img_inference(img_infer)

In [ ]:
%cd /content/keras-retinanet